# SEQ 2D

Schrödinger equation is given as:

$$
(-\frac{\hbar^2}{2m} \Delta  - \frac{Ze^2}{r - r_d}) \Psi = E \Psi(x,y) 
$$

Laplas operator

$$
\Delta = \frac{\partial^2}{\partial^2 x} + \frac{\partial^2}{\partial^2 y} \\
\frac{\partial^2 \Psi}{\partial^2 x} = \frac{\Psi(x+1) - 2\Psi(x) + \Psi(x-1)}{dx^2}  \\
\frac{\partial^2 \Psi}{\partial^2 y} = \frac{\Psi(y+1) - 2\Psi(y) + \Psi(x-1)}{dy^2}  \\
$$

Let's suppose that we have (3x3) grid. Wave function is represented as:
$$
\begin{pmatrix}
\Psi_{11} & \Psi_{12} & \Psi_{13}  \\
\Psi_{21} & \Psi_{22} & \Psi_{23}  \\
\Psi_{31} & \Psi_{32} & \Psi_{33}  \\
\end{pmatrix}
$$

We will present it like vector:
$$
\begin{pmatrix}
\Psi_{11} & \Psi_{12} & \Psi_{13} &  \Psi_{21} & \Psi_{22} & \Psi_{23} & \Psi_{31} & \Psi_{32} & \Psi_{33} \\ 
\end{pmatrix}^T
$$

Within this basis differential operator x will have the followig form:
$$
\frac{\partial^2 }{\partial^2 y} = 
\frac{1}{dx^2}
\begin{pmatrix}
-2 & 1 & 0 & 0 & 0 & 0 &   0 & 0 & 0 \\
1 & -2 & 1 & 0 & 0 & 0 &  0 & 0 & 0  \\
0 &  1 & -2 & 0 & 0 & 0 & 0 & 0 & 0  \\
0 & 0 & 0 & -2 & 1 & 0 &   0 & 0 & 0 \\
0 & 0 & 0 & 1 & -2 & 1 &   0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & -2 &   0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 &   -2 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 &   1 & -2 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 &   0 & 1 & -2 \\
\end{pmatrix} = 
\frac{1}{dx^2}
\begin{pmatrix}
-2 & 1 & 0 \\
1 & -2 & 1 \\
0 &  1 & -2 \\
\end{pmatrix} \otimes \hat{I}  = \frac{1}{dx^2} D \otimes \hat{I} 
\\
\\
$$
On the other hand, differential operator y 
$$
\frac{\partial^2 }{\partial^2 x} = 
\frac{1}{dy^2}
\begin{pmatrix}
-2\hat{I} & 1\hat{I} & 0 \\
\hat{I} & -2\hat{I} & \hat{I} \\
0 &  \hat{I} & -2\hat{I} \\
\end{pmatrix} = 
\frac{1}{dy^2}
\begin{pmatrix}
-2 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & -2 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & -2 & 0 & 0 & 1 & 0 & 0 & 0 \\
1 & 0 & 0 & -2 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & -2 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & -2 & 0 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 & 0 & -2 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & -2 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & -2 \\
\end{pmatrix}  = \frac{1}{dy^2}  \hat{I} \otimes D
$$

Resulting Laplas operator will have the form: $$ (dx = dy = d)$$
$$
\Delta = \frac{\partial^2}{\partial^2 x} + \frac{\partial^2}{\partial^2 y} = \frac{1}{d^2} ( D \otimes \hat{I} +  \hat{I} \otimes D) = \frac{1}{d^2} D \otimes D
$$

The final Schrödinger equation takes the form:

$$
(-\frac{1}{2m d^2} D \otimes D  - \frac{Ze^2}{r - r_d}) \Psi = E \Psi(x,y) 
$$


In [ ]:
import numpy as np
import scipy as sp
import torch
import time 
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib.colors as colors


def get_potential_ring(x, y, scr_length):
    d  = 4
    delta = 1
    r = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x) 
    V = (-1)/(np.abs(r - 0.1 * d * np.cos(phi)**2 - d) + delta)*np.exp(-np.abs(r - 0.1 * d * np.cos(phi)**2 - d)/scr_length) 
    return V



def get_potential_points(x, y, scr_length):
    d  = 4
    delta = 1
    coord = np.array([[-d, 0], [d, 0], [0, -d], [0, d], [d/np.sqrt(2), d/np.sqrt(2)], [d/np.sqrt(2), -d/np.sqrt(2)], [-d/np.sqrt(2), d/np.sqrt(2)], [-d/np.sqrt(2), -d/np.sqrt(2)]])
    V = 0
    for i in range(8):
        r_i = np.sqrt((x - coord[i][0])**2 + (y - coord[i][1])**2)
        V -= 0.8/(r_i + delta)*np.exp(-r_i/scr_length)
    return V

num_points = 80
scr_length = 10
rMax = 10

results_dir = 'results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    

line = np.logspace(0, np.log10(rMax), num=int(0.5 * num_points))
line_tot = np.concatenate((-np.flip(line), line))


X, Y = np.meshgrid(line_tot,line_tot) # in Ang
V = get_potential_ring(X, Y, scr_length)

fig = plt.figure(figsize=(5,5))
ax = plt.subplot()
im = ax.pcolor(X, Y, V, shading='auto', cmap='inferno')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
fig.savefig(os.path.join(results_dir, 'Potential.png'), dpi=200, facecolor='w', transparent=False, bbox_inches='tight')


In [ ]:
diag = np.ones([num_points])
diags = np.array([diag, -2 * diag, diag])
D = sp.sparse.spdiags(diags, np.array([-1, 0, 1]), num_points, num_points)
T = -(1/2) * sp.sparse.kronsum(D, D)

U = sparse.diags(V.reshape(num_points**2),(0))
Ham = T + U

print('hamiltonian was constructed!')

In [ ]:
#eigenproblem

tick = time.time()
# evals, evecs = sp.sparse.linalg.eigsh(Ham, k = 2, which = 'SM')

# Ham_dense = Ham.todense()
# evals, evecs = np.linalg.eigh(Ham_dense)


# Ham_dense = Ham.todense()
# with torch.no_grad():
#     w, v = torch.linalg.eigh(torch.from_numpy(Ham_dense))
#     evals = w.numpy()
#     evecs = v.numpy()
    

tock = time.time()
print("Diagonalization time:", tock - tick, 'sec')

In [ ]:
m = 2

def get_e(num):
    return np.array(evecs[:,num].reshape((num_points, num_points)))

fig = plt.figure(figsize=(5,5))
ax = plt.subplot()
plt.title(f'eigenvalue = {str(evals[m])}',fontsize=12)
im = ax.pcolor(X, Y, get_e(m)**2)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
fig.savefig(os.path.join(results_dir, f'Eigenstate_{str(m)}.png'), dpi=200, facecolor='w', transparent=False, bbox_inches='tight')